In [1]:
import matplotlib.pyplot as plt
import vcdvcd

# Load and parse the VCD file
vcd_path = "/mnt/data/i2c.vcd"
vcd = vcdvcd.VCDVCD(vcd_path, signals=None, store_tvs=True)

# Extract signals from the VCD
signals = list(vcd.data.keys())
signals_to_plot = signals[:4]  # Select a few signals to visualize

# Plot signal waveforms
plt.figure(figsize=(12, 6))
for i, signal in enumerate(signals_to_plot):
    tvs = vcd.data[signal]['tv']
    times = [tv[0] for tv in tvs]
    values = [int(tv[1]) for tv in tvs]
    plt.step(times, [v + 2*i for v in values], where='post', label=signal)

plt.yticks(range(0, 2*len(signals_to_plot), 2), [sig.split(' ')[-1] for sig in signals_to_plot])
plt.xlabel("Time (ns)")
plt.ylabel("Signal")
plt.title("Waveform for Selected I2C Signals")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'vcdvcd'

In [ ]:
# Manually parse basic VCD signals for waveform visualization
def parse_vcd_signals(file_path, signal_names, max_time_ns=2000):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    signal_map = {}
    in_dumpvars = False
    for line in lines:
        if line.startswith("$var"):
            parts = line.strip().split()
            symbol = parts[3]
            name = parts[4]
            if name in signal_names:
                signal_map[symbol] = {'name': name, 'values': []}
        if line.startswith("$dumpvars"):
            in_dumpvars = True
        elif line.startswith("$enddefinitions"):
            break

    current_time = 0
    for line in lines:
        if line.startswith('#'):
            current_time = int(line[1:])
            if current_time > max_time_ns:
                break
        elif line and line[0] in '01':
            val = int(line[0])
            sym = line[1:].strip()
            if sym in signal_map:
                signal_map[sym]['values'].append((current_time, val))

    return signal_map

# Parse signals of interest
signal_names = ['clk', 'rst', 'scl', 'sda']
vcd_signals = parse_vcd_signals("/mnt/data/i2c.vcd", signal_names)

# Plot the waveform
plt.figure(figsize=(12, 6))
for idx, (symbol, signal_data) in enumerate(vcd_signals.items()):
    times, values = zip(*signal_data['values'])
    shifted_values = [v + 2 * idx for v in values]
    plt.step(times, shifted_values, where='post', label=signal_data['name'])

plt.yticks([2 * i for i in range(len(vcd_signals))], [data['name'] for data in vcd_signals.values()])
plt.xlabel("Time (ns)")
plt.ylabel("Signal Level")
plt.title("I2C Waveform Visualization")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
